In [1]:
import numpy as np
import pandas as pd
import generate_image_labels as gil
import image_processing as ip

from PIL import Image

import matplotlib.pyplot as plt

plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = ( 14, 10 )

%matplotlib inline

In [2]:
# Need to go line by line, processing images
def process_pipeline(
                        inp_df,
                        new_img_size,
                    ):
    
    my_df = inp_df.copy()
    
    # Create df for storing pixel values
    pixel_df = pd.DataFrame( 
                                0,
                            index=my_df.index.values,
                            columns=[ 'pixel_'+str(i) for i in range( 0, new_img_size[0]*new_img_size[1] ) ],
                            dtype=float
                           )
        
    for i in range( 0, my_df.shape[0] ):

        ind = my_df.index.values[i]

        # First time through, read from file, convert to greyscale
        # Second time, pass image back through    
        conv_grey = True
        pi_inp    = my_df.loc[ind,'img_path']

        # Do however many processes
        for j in range( 0, len(RESIZE_IMAGE) ):
                            
            # If we are doing multiple passes, return
            #  a PIL image, except the last time
            ret_pil = False
            if ( 
                ( len(RESIZE_IMAGE)     > 1 ) and 
                (     RESIZE_IMAGE[j]-1 > j )
               ):
                ret_pil = True
                
                
            img_arr = ip.process_image(
                                        pi_inp,
                                        resize_image   = RESIZE_IMAGE[j],
                                        resize_shape   = RESIZE_SHAPE[j],
                                        blur_image     = BLUR_IMAGE[j],
                                        kernel_size    = KERNEL_SIZE[j],
                                        pool_image     = POOL_IMAGE[j],
                                        pool_stride    = POOL_STRIDE[j],
                                        pool_grid_size = POOL_GRID_SIZE[j],
                                        pool_method    = POOL_METHOD[j],
                                        conv_greyscale = conv_grey,
                                        ret_pil_img    = ret_pil,
                                      )
            
            # Any next pases will not need greyscale
            #  and input will change to pil image
            conv_grey = False
            try:
                pi_inp    = Image.fromarray( img_arr.astype(np.uint8) )
            except:
                pass
        
        # Fill the pixel arrray
        pixels_1d = img_arr.reshape( 1, -1 )
        pixel_df.loc[ind] = pixels_1d
        
    return pd.concat( [my_df, pixel_df], axis=1 )

def flip_rot(
                inp_df,
                img_size
            ):
    
    pixel_list   = [ col for col in inp_df.columns.values if ('pixel_'     in col ) ]
    not_pix_list = [ col for col in inp_df.columns.values if ('pixel_' not in col ) ]
    
    # Create df for storing pixel values
    pixel_df = pd.DataFrame( 
                             columns=not_pix_list+['rotated','transposed']+pixel_list,
                             dtype=object
                           )
    

    new_ind_counter = 0
    
    
    for ind in inp_df.index.values:
    
        # Loop over input, flipping the array values
        img_arr = np.array( 
                            inp_df.loc[0,pixel_list].values
                          ).reshape( 
                                        img_size[0], 
                                        img_size[1] 
                                   )

        # Flip array and dont
        for flip in [False,True]:
            if ( flip ):
                img_arr = ip.h_flip_array( img_arr )

            # Each image rotation
            for rot in range( 1, 5 ):
                img_arr = ip.rotate_array( img_arr, 1 )

                # Write to new list of values
                new_vals = (
                                inp_df.loc[ind,not_pix_list].values.tolist() + 
                                [rot,int(flip)] +
                                img_arr.reshape( 1, -1 ).tolist()[0]
                           )


                pixel_df.loc[new_ind_counter] = new_vals
                new_ind_counter = new_ind_counter + 1

    return pixel_df


# Gets 2d arrays out of the pixels in the df
def arr_list_from_pix_df(
                            inp_df,
                            inp_size
                        ):
    
    pixel_cols = [ col for col in inp_df.columns.values if ('pixel_'     in col ) ]

    img_list = []
    
    for ind in inp_df.index.values:
        arr_1d = inp_df.loc[ind,pixel_cols].values
        img_list.append( np.array( arr_1d ).reshape(inp_size[0],inp_size[1]) )
        
    return img_list

In [3]:
RESIZE_IMAGE = [ 
                    True, 
                    True, 
                    True, 
               ]
RESIZE_SHAPE = [ 
                     [1024,1024],
                     [512,512],
                     [256,256],
               ]

BLUR_IMAGE  = [ 
                    True, 
                    True, 
                    True, 
              ]
KERNEL_SIZE = [
                5,
                5,
                3,
              ]  

POOL_IMAGE     = [ 
                    True, 
                    True, 
                    True, 
                 ]

POOL_STRIDE    = [ 
                    4, 
                    4,
                    4,
                 ]
POOL_GRID_SIZE = [ 
                    4, 
                    4,
                    4,
                 ]
POOL_METHOD    = [ 
                    'max', 
                    'max',
                    'max',
                 ]

In [4]:
img_df = gil.gen_white_labels()

pro = process_pipeline( 
                        img_df, 
                        [
                            RESIZE_SHAPE[-1][0]/POOL_STRIDE[-1],
                            RESIZE_SHAPE[-1][0]/POOL_STRIDE[-1],
                        ] 
                      )

boosted = flip_rot( 
                        pro, 
                        [
                            RESIZE_SHAPE[-1][0]/POOL_STRIDE[-1],
                            RESIZE_SHAPE[-1][0]/POOL_STRIDE[-1],
                        ] 
                      )

boosted.to_csv('data/white_3_1024_516_256_5_5_3_4_4_4.csv')

/home/sean/anaconda/lib/python2.7/site-packages/PIL/Image.py:918: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


NameError: name 'foo' is not defined

In [6]:
boosted.head()

,img_path,label,height_brick,height_plate,height_other,shape_corner,shape_round,shape_square,short_1,short_2,...,pixel_4086,pixel_4087,pixel_4088,pixel_4089,pixel_4090,pixel_4091,pixel_4092,pixel_4093,pixel_4094,pixel_4095
0,/home/sean/Desktop/lego_images_bounding_labele...,brick_corner_1x2x2,1,0,0,1,0,0,1,0,...,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0
1,/home/sean/Desktop/lego_images_bounding_labele...,brick_corner_1x2x2,1,0,0,1,0,0,1,0,...,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0
2,/home/sean/Desktop/lego_images_bounding_labele...,brick_corner_1x2x2,1,0,0,1,0,0,1,0,...,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0
3,/home/sean/Desktop/lego_images_bounding_labele...,brick_corner_1x2x2,1,0,0,1,0,0,1,0,...,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0
4,/home/sean/Desktop/lego_images_bounding_labele...,brick_corner_1x2x2,1,0,0,1,0,0,1,0,...,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0


In [7]:
boosted.tail()

,img_path,label,height_brick,height_plate,height_other,shape_corner,shape_round,shape_square,short_1,short_2,...,pixel_4086,pixel_4087,pixel_4088,pixel_4089,pixel_4090,pixel_4091,pixel_4092,pixel_4093,pixel_4094,pixel_4095
2779,/home/sean/Desktop/lego_images_bounding_labele...,arch_1x6x2,0,0,1,0,0,1,1,0,...,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0
2780,/home/sean/Desktop/lego_images_bounding_labele...,arch_1x6x2,0,0,1,0,0,1,1,0,...,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0
2781,/home/sean/Desktop/lego_images_bounding_labele...,arch_1x6x2,0,0,1,0,0,1,1,0,...,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0
2782,/home/sean/Desktop/lego_images_bounding_labele...,arch_1x6x2,0,0,1,0,0,1,1,0,...,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0
2783,/home/sean/Desktop/lego_images_bounding_labele...,arch_1x6x2,0,0,1,0,0,1,1,0,...,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0
